In [1]:
import pandas as pd
from neo4j import GraphDatabase

In [2]:
uri = "neo4j://localhost:7687"
conn = GraphDatabase.driver(uri, auth=("neo4j", "Ass2DB"), max_connection_lifetime=1000)

# Articles

In [12]:
art = pd.read_csv('Citation Dataset\\ArticleNodes.csv', header=None)#, nrows=1000)
art.columns = ['id','title','year','journal','abstract']
art.head() # 29555

,id,title,year,journal,abstract
0,1001,Compactification Geometry and Duality: N=2,2000,NaN,These are notes based on lectures given at T...
1,1002,Domain Walls and Massive Gauged Supergravity P...,2000,Class.Quant.Grav.,We point out that massive gauged supergravit...
2,1003,Comment on Metric Fluctuations in Brane Worlds,2000,NaN,Recently- Ivanov and Volovich (hep-th/991224...
3,1004,Moving Mirrors and Thermodynamic Paradoxes,2000,Phys.Rev.,Quantum fields responding to moving mirrors...
4,1005,Bundles of chiral blocks and boundary conditio...,2000,NaN,Various aspects of spaces of chiral blocks a...


In [13]:
art[art['id'] == 9304045]

,id,title,year,journal,abstract
12471,9304045,Generalized Calabi-Yau Manifolds and the Mirro...,1993,Nucl.Phys.,We describe the mirror of the Z orbifold as ...


### Create Article Nodes

In [7]:
for index, row in art.iterrows():
    article =  'ID' + str(row['id'])
    title = str(row['title']).replace('"','').replace("'",'').replace('`','').replace('{','').replace('}','').replace('\\','').replace('/','')
    abst = str(row['abstract']).replace('"','').replace("'",'').replace('`','').replace('{','').replace('}','').replace('\\','').replace('/','')
    
    cqlCreate = "CREATE ({}:article".format(article) + " {" + " id: "'"{}"'", title: "'"{}"'", year: "'"{}"'", journal: "'"{}"'", abstract: "'"{}"'"".format(article, title, row['year']
                    , str(row['journal']) , abst ) + "})"    # cont of .formt( then close out

    with conn.session() as graphDB_Session:
        # Create nodes
        graphDB_Session.run(cqlCreate)

## Citation Relationships

In [8]:
cit = pd.read_csv('Citation Dataset\\Citations.csv', sep = '\t', header=None)
cit.columns = ['id1','id2']
cit.head()

,id1,id2
0,1001,9304045
1,1001,9308122
2,1001,9309097
3,1001,9311042
4,1001,9401139


In [9]:
for index, row in cit.iterrows():
    id1 =  'ID' + str(row['id1'])
    id2 =  'ID' + str(row['id2'])

    cqlCreate = "CREATE ({})-[:cites]->({})".format(id1,id2)

    with conn.session() as graphDB_Session:
        # Create nodes
        graphDB_Session.run(cqlCreate)

StopIteration: 

# MANUAL
1001	Paul S. Aspinwall	Compactification Geometry and Duality: N=2	2000	NaN

9304045	Generalized Calabi-Yau Manifolds and the Mirro...	1993	Nucl.Phys.

In [3]:
cqlCreate = "CREATE (ID1001:article { id: 'ID1001' , title: 'Compactification Geometry and Duality: N=2', year: '2000', journal: 'NaN'})"    # cont of .formt( then close out

with conn.session() as graphDB_Session:
    # Create nodes
    graphDB_Session.run(cqlCreate)

In [4]:
cqlCreate = "CREATE (ID9304045:article { id: 'ID9304045', title: 'Generalized Calabi-Yau Manifolds', year: '1993', journal: 'Nucl.Phys.'})"    # cont of .formt( then close out

with conn.session() as graphDB_Session:
    # Create nodes
    graphDB_Session.run(cqlCreate)

In [5]:
cqlCreate = "CREATE (ID1001)-[:cites]->(ID9304045)"

with conn.session() as graphDB_Session:
    # Create nodes
    graphDB_Session.run(cqlCreate)

In [ ]:
# Match relationship

MATCH (a:article), (b:article) WHERE a.id = "ID1001" AND b.id = "ID9304045" 
CREATE (a)-[r: cites]->(b) 
RETURN a,b 

In [ ]:
"""
CREATE 
(ID1001:article { id: 'ID1001' , title: 'Compactification Geometry and Duality: N=2', year: '2000', journal: 'NaN'})
,
(ID9304045:article { id: 'ID9304045', title: 'Generalized Calabi-Yau Manifolds', year: '1993', journal: 'Nucl.Phys.'})
,
(ID1001)-[:cites]->(ID9304045)


"""

# Authors

In [4]:
aut = pd.read_csv('Citation Dataset\\AuthorNodes.csv', header=None)
aut = aut.reset_index()
aut.columns = ['autid','artid','author']
aut.head() 

,id,author
0,1001,Paul S. Aspinwall
1,1002,C.N. Pope
2,1002,M. Cvetic
3,1002,H. Lu
4,1003,Y.S. Myung


### Create Author Nodes

In [6]:
for index, row in aut.iterrows():
    authorid =  'ID' + str(row['autid'])
    articleid =  'ID' + str(row['artid'])
    author = str(row['author']).replace('"','').replace("'",'').replace('`','').replace('{','').replace('}','').replace('\\','').replace('/','')

    cqlCreate = "CREATE ({}:author".format(authorid) + " {" + " id: "'"{}"'", author: "'"{}"'"".format(articleid, author) + "})"    # cont of .formt( then close out

    with conn.session() as graphDB_Session:
        # Create nodes
        graphDB_Session.run(cqlCreate)

## Author Relationships

In [ ]:
for index, row in aut.iterrows():
    id1 =  'ID' + str(row['autid'])
    id2 =  'ID' + str(row['artid'])

    #cqlCreate = "CREATE ({})-[:writes]->({})".format(id1,id2)
    
    cqlCreate = """
    
    MATCH (a:author), (b:article) WHERE a.id = "ID101" AND b.id = "IDwhatever" 
    CREATE (a)-[r: writes]->(b) 
    RETURN a,b 
    
    """

    with conn.session() as graphDB_Session:
        # Create nodes
        graphDB_Session.run(cqlCreate)

In [10]:
writes = pd.DataFrame(aut['id'])
writes.columns = ['id1']
writes['id2'] = writes['id1']
writes

,id1,id2
0,1001,1001
1,1002,1002
2,1002,1002
3,1002,1002
4,1003,1003
...,...,...
58335,9912292,9912292
58336,9912292,9912292
58337,9912293,9912293
58338,9912294,9912294


In [ ]:




MATCH (a:author), (b:article) WHERE a.id = "ID101" AND b.id = "IDwhatever" 
CREATE (a)-[r: writes]->(b) 
RETURN a,b 



In [ ]:
# CQLrelate = """

# MATCH
#   ({}:article),
#   ({}:article)
# WHERE {}.id = '{}' AND {}.name = '{}'
# CREATE ({})-[r:cites]->({})
# RETURN type(r)

# """.format(  id1, id2,  id1, id1,    id2, id2,    id1,id2  )

In [ ]:
for index, row in cit.iterrows():
    id1 =  'ID' + str(row['id1'])
    id2 =  'ID' + str(row['id2'])

    cqlCreate = "CREATE ({})-[:cites]->({})".format(id1,id2)

    with conn.session() as graphDB_Session:
        # Create nodes
        graphDB_Session.run(cqlCreate)